In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import Parameter
from torch.nn.utils import clip_grad_norm

import torchtext
from torchtext import *
from fastai.text import *
from fastai.callbacks import *

import pandas as pd
import random
from itertools import chain
import math
import numpy as np
import time

In [0]:
def random_seed(seed_value, use_cuda):
  np.random.seed(seed_value) # cpu vars
  torch.manual_seed(seed_value) # cpu  vars
  random.seed(seed_value) # Python
  if use_cuda:
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value) # gpu vars
    torch.backends.cudnn.deterministic = True  #needed
    torch.backends.cudnn.benchmark = False
random_seed(180127818, True)
#torch.cuda.set_device(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
# Wikitext-2
path = '/content/drive/My Drive/wikitext-2/'
#epoch number, follow https://github.com/fastai/fastai/blob/master/examples/ULMFit.ipynb
lm_epoch = 10
cls_epoch = 2
bptt=70
bs=32

In [5]:
%cd /content/drive/My\ Drive/

/content/drive/My Drive


In [6]:
!wget https://s3.amazonaws.com/fast-ai-nlp/wikitext-2.tgz
!tar -zxvf wikitext-2.tgz

--2019-11-21 20:25:37--  https://s3.amazonaws.com/fast-ai-nlp/wikitext-2.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.95.69
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.95.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4070055 (3.9M) [application/x-tar]
Saving to: ‘wikitext-2.tgz.2’

wikitext-2.tgz.2    100%[===================>]   3.88M  7.68MB/s    in 0.5s    

2019-11-21 20:25:38 (7.68 MB/s) - ‘wikitext-2.tgz.2’ saved [4070055/4070055]

wikitext-2/
wikitext-2/train.csv
wikitext-2/test.csv


In [0]:
train=pd.read_csv('./wikitext-2/train.csv',header=None,sep='/n/t/n/t', engine='python')
test=pd.read_csv('./wikitext-2/test.csv',header=None,sep='/n/t/n/t', engine='python')

In [0]:
# CREATE data
# Language model data bunch
data_lm = TextLMDataBunch.from_df('.',train_df=train,valid_df=test,text_cols=0)
data_lm.save(path+'data_lm.pkl')

In [0]:
# LOAD data & learner
#forward
data_lm = load_data(path, 'data_lm.pkl', bs=bs, bptt=bptt)
#bwd
#data_bwd = load_data(path, 'data_lm.pkl', bs=128, bptt=70, backwards=True)

In [10]:
learn = language_model_learner(data_lm, AWD_LSTM, pretrained=True)
# learn = learn.to_fp16(clip=0.1)
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7), wd=0.1)
#learn_bwd = language_model_learner(data_bwd, AWD_LSTM, pretrained=True)
#learn_bwd = learn_bwd.to_fp16(clip=0.1)
#learn_bwd.fit_one_cycle(1, 1e-2, moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.783408,3.418671,0.364780,01:39


In [0]:
learn.save('learn')

In [12]:
learn = language_model_learner(data_lm, AWD_LSTM, pretrained=False)
learn.load('learn')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (24974 items)
x: LMTextList
xxbos ",xxbos = 2013 – 14 xxmaj york xxmaj city xxup f.c. season =,xxbos xxmaj the 2013 – 14 season was the xxunk season of competitive association football and 77th season in the xxmaj football xxmaj league played by xxmaj york xxmaj city xxmaj football xxmaj club , a professional football club based in xxmaj york , xxmaj north xxmaj yorkshire , xxmaj england . xxmaj their 17th - place finish in 2012 – 13 meant it was their second consecutive season in xxmaj league xxmaj two . xxmaj the season ran from 1 xxmaj july 2013 to 30 xxmaj june 2014 .,xxbos xxmaj nigel xxmaj worthington , starting his first full season as xxmaj york manager , made eight permanent summer signings . xxmaj by the turn of the year xxmaj york were only above the relegation zone on goal difference , before a 17-match unbeaten run saw the team finish in seventh - place in the xxunk 2013 – 14 xxmaj football xxmaj league xxmaj two . xx

In [0]:
class LinearSVDO(nn.Module):
  def __init__(self, in_features, out_features):
    super(LinearSVDO, self).__init__()
    self.is_training = 1
    self.in_features = in_features
    self.out_features = out_features
    self.weight = Parameter(torch.Tensor(out_features, in_features))
    self.log_sigma = Parameter(torch.Tensor(out_features, in_features))
    self.bias = Parameter(torch.Tensor(1, out_features))
    self.reset_parameters()
  
  def reset_training(self, v=0):
    self.is_training = v

  def reset_parameters(self):
    self.bias.data.zero_()
    self.weight.data.normal_(0, 0.02)
    self.log_sigma.data.fill_(-5)        
        
  def forward(self, x):
    self.log_alpha = self.log_sigma * 2.0 - 2.0 * torch.log(1e-16 + torch.abs(self.weight))
    self.log_alpha = torch.clamp(self.log_alpha, -10, 10) 
    lrt_mean =  F.linear(x, self.weight) + self.bias    
    if self.training: 
      lrt_std = torch.sqrt(F.linear(x * x, torch.exp(self.log_sigma * 2.0)) + 1e-8)
      eps = lrt_std.data.new(lrt_std.size()).normal_()
      return lrt_mean + lrt_std * eps
    return lrt_mean
        
  def kl_reg(self):
    # Return KL here -- a scalar 
    k1, k2, k3 = torch.Tensor([0.63576]).to(device), torch.Tensor([1.8732]).to(device), torch.Tensor([1.48695]).to(device)
    kl = k1 * torch.sigmoid(k2 + k3 * self.log_alpha) - 0.5 * torch.log1p(torch.exp(-self.log_alpha))
    a = - torch.sum(kl)
    return a

In [0]:
class LSTMSVDO(nn.Module):
  def __init__(self, input_sz: int, hidden_sz: int):
    super().__init__()
    self.is_training = 1
    self.input_sz = input_sz
    self.hidden_size = hidden_sz
    self.weight_ih = Parameter(torch.Tensor(hidden_sz * 4, input_sz))
    self.weight_hh = Parameter(torch.Tensor(hidden_sz * 4, hidden_sz))
    self.log_sigma_ih = Parameter(torch.Tensor(hidden_sz * 4, input_sz))
    self.log_sigma_hh = Parameter(torch.Tensor(hidden_sz * 4, hidden_sz))
    self.bias = Parameter(torch.Tensor(hidden_sz * 4))
    self.reset_parameters()
  
  def reset_training(self, v=0):
    self.is_training = v
    
  def reset_parameters(self):
    self.bias.data.zero_()
    self.weight_ih.data.normal_(0, 0.02)
    self.weight_hh.data.normal_(0, 0.02)
    self.log_sigma_ih.data.fill_(-5)
    self.log_sigma_hh.data.fill_(-5)

  def forward(self, x: torch.Tensor, init_states=None):
    """Assumes x is of shape (batch, sequence, feature)"""
    self.log_alpha_ih = torch.clamp(self.log_sigma_ih * 2.0 - 2.0 * torch.log(1e-16 + torch.abs(self.weight_ih)), -10, 10) 
    self.log_alpha_hh = torch.clamp(self.log_sigma_hh * 2.0 - 2.0 * torch.log(1e-16 + torch.abs(self.weight_hh)), -10, 10) 
    bs, seq_sz, _ = x.size()
    hidden_seq = []
    if init_states is None:
      h_t, c_t = (torch.zeros(self.hidden_size).to(x.device), torch.zeros(self.hidden_size).to(x.device))
    else:
      h_t, c_t = init_states 
    HS = self.hidden_size
    for t in range(seq_sz):
      x_t = x[:, t, :]
      # batch the computations into a single matrix multiplication
      if self.is_training:
        lrt_std_ih = torch.sqrt(F.linear((x_t * x_t), torch.exp(self.log_sigma_ih * 2.0)) + 1e-8)
        lrt_std_hh = torch.sqrt(F.linear((h_t * h_t), torch.exp(self.log_sigma_hh * 2.0)) + 1e-8)
        eps_ih = lrt_std_ih.data.new(lrt_std_ih.size()).normal_()
        eps_hh = lrt_std_hh.data.new(lrt_std_hh.size()).normal_()

        gates = F.linear(x_t, self.weight_ih) + F.linear(h_t, self.weight_hh) + self.bias + lrt_std_ih * eps_ih + lrt_std_hh * eps_hh
      else:
        gates = F.linear(x_t, self.weight_ih) + F.linear(h_t, self.weight_hh) + self.bias
      i_t, f_t, g_t, o_t = (
          torch.sigmoid(gates[:, :HS]), # input
          torch.sigmoid(gates[:, HS:HS*2]), # forget
          torch.tanh(gates[:, HS*2:HS*3]),
          torch.sigmoid(gates[:, HS*3:]), # output
          )
      c_t = f_t * c_t + i_t * g_t
      h_t = o_t * torch.tanh(c_t)
      hidden_seq.append(h_t.unsqueeze(0))
    hidden_seq = torch.cat(hidden_seq, dim=0)
    h_t, c_t = h_t.detach(), c_t.detach()
    # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
    hidden_seq = hidden_seq.transpose(0, 1).contiguous()
    return hidden_seq#, (h_t, c_t)

          
  def kl_reg(self):
    # Return KL here -- a scalar 
    a_sum = torch.Tensor([0.]).to(device)
    k1, k2, k3 = torch.Tensor([0.63576]).to(device), torch.Tensor([1.8732]).to(device), torch.Tensor([1.48695]).to(device)
    for log_alpha in [self.log_alpha_ih, self.log_alpha_hh]:
      kl = k1 * torch.sigmoid(k2 + k3 * log_alpha) - 0.5 * torch.log1p(torch.exp(-log_alpha))
      a = - torch.sum(kl)
      a_sum = a_sum + a
    return a_sum

In [0]:
# Define New Loss Function -- SGVLB 
class SGVLB(nn.modules.loss.Module):
  def __init__(self, net, train_size):
    super(SGVLB, self).__init__()
    self.train_size = train_size
    self.net = net

  def forward(self, input, target, kl_weight=1.0):
    assert not target.requires_grad
    kl = 0.0
    for module in [self.net.rnn1, self.net.rnn2, self.net.rnn3, self.net.decoder]:#for module in [self.net.rnn1, ]:
      if hasattr(module, 'kl_reg'):
        kl = kl + module.kl_reg()
    return F.nll_loss(input, target) + kl_weight * kl / self.train_size#F.nll_loss(input, target) * self.train_size + kl_weight * kl / bs #cross_entropy

In [0]:
# Define a simple 3 layer LSTM
class Net(nn.Module):
  def __init__(self, vocab_size, num_embed, num_hidden, **kwargs):
    super(Net, self).__init__()
    self.embedding = nn.Embedding(vocab_size, num_embed, padding_idx=1)
    self.rnn1 = LSTMSVDO(num_embed, num_hidden)
    self.rnn2 = LSTMSVDO(num_hidden, num_hidden)
    self.rnn3 = LSTMSVDO(num_hidden, num_embed)
    self.decoder = LinearSVDO(num_embed, vocab_size)
  
  def reset_training(self, v):
    for i in [self.rnn1, self.decoder]:#[self.rnn1, self.rnn2, self.rnn3, self.decoder]:
      i.reset_training(v)

  def forward(self, x: torch.Tensor):
    """Assumes x is of shape (batch, sequence, feature)"""
    x = self.embedding(x)
    #init_states = init_states
    #x, init_states[0] = self.rnn1(x, init_states[0])
    #x, init_states[1] = self.rnn2(F.relu(x), init_states[1])
    #x, init_states[2] = self.rnn3(F.relu(x), init_states[2])
    x = self.rnn1(x)
    x = self.rnn2(F.relu(x))
    x = self.rnn3(F.relu(x))
    # Reshape output to (batch_size*sequence_length, hidden_size)
    x = F.log_softmax(self.decoder(F.relu(x).reshape(x.size(0)*x.size(1),x.size(2))))
    #x = F.log_softmax(self.decoder(x[:,-1,:]))
    return x

In [0]:
model = Net(vocab_size=len(data_lm.train_ds.vocab.itos), num_embed=400, num_hidden=1152).to(device)

In [0]:
model.embedding.weight.data = learn.model[0].encoder.weight.data
model.rnn1.weight_ih.data = learn.model[0].rnns[0].module.weight_ih_l0.data
model.rnn1.weight_hh.data = learn.model[0].rnns[0].module.weight_hh_l0.data
model.rnn1.bias.data = learn.model[0].rnns[0].module.bias_ih_l0.data
model.rnn2.weight_ih.data = learn.model[0].rnns[1].module.weight_ih_l0.data
model.rnn2.weight_hh.data = learn.model[0].rnns[1].module.weight_hh_l0.data
model.rnn2.bias.data = learn.model[0].rnns[1].module.bias_ih_l0.data
model.rnn3.weight_ih.data = learn.model[0].rnns[2].module.weight_ih_l0.data
model.rnn3.weight_hh.data = learn.model[0].rnns[2].module.weight_hh_l0.data
model.rnn3.bias.data = learn.model[0].rnns[2].module.bias_ih_l0.data
model.decoder.weight.data = learn.model[1].decoder.weight.data
model.decoder.bias.data = learn.model[1].decoder.bias.data

In [0]:
def create_lmdata(data):
    ids = torch.LongTensor(list(chain(*data)))
    num_batches = ids.size(0) // bs
    ids = ids[:num_batches*bs]
    ids = ids.view(bs, -1)
    return ids
ids = create_lmdata(data_lm.train_ds.x.items)
valid_ids = create_lmdata(data_lm.valid_ds.x.items)

In [0]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,60,70,80], gamma=0.2)
sgvlb = SGVLB(model, ids.shape[1])

In [0]:
def myeval(ids=valid_ids):
    # use in testing time, so use is_traing=0
    total_L = 0.
    model.reset_training(0)
    criterion = nn.modules.loss.NLLLoss()
    for i in range(0, ids.size(1) - bptt, bptt):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+bptt].to(device)
        targets = ids[:, (i+1):(i+1)+bptt].to(device)
        output = model(inputs)
        loss = criterion(output, targets.reshape(-1)).item()
        total_L += loss
    model.reset_training(1)
    return total_L

In [0]:
criterion = nn.modules.loss.NLLLoss()

In [0]:
def train(kl_weight=0.02, epochs=100): 
    kl_weight = kl_weight
    epochs = epochs
    for epoch in range(1, epochs + 1):
        start_time = time.time()
        train_loss, train_acc = 0, 0 
        kl_weight = min(kl_weight+0.02,1)
        init_states = None
        for i in range(0, ids.size(1) - bptt, bptt):
            inputs = ids[:, i:i+bptt].to(device)
            targets = ids[:, (i+1):(i+1)+bptt].to(device)
            output = model(inputs)
            #loss = criterion(output, targets.reshape(-1))
            loss = sgvlb(output, targets.reshape(-1), kl_weight)
            model.zero_grad()
            loss.backward()
            clip_grad_norm(model.parameters(), 0.5)
            optimizer.step()
            #if i % 100 == 0:
                #print ('Epoch [{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'.format(epoch, epochs, loss.item(), np.exp(loss.item())))
        L = myeval(ids=valid_ids)
        scheduler.step()
        print('Epoch [{}/{}], Valid_Loss: {:.4f}, Perplexity: {:5.2f}, Time: {:.2f}'.format(epoch, epochs, L, np.exp(L), time.time()-start_time))
        
        if epoch %10==1:
            torch.save(model.state_dict(), path+'model_'+str(epoch)+'.pkl')



In [24]:
model.rnn1.reset_training(1)
model.decoder.reset_training(1)
train(kl_weight=0.02, epochs=100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch [1/100], Valid_Loss: 470.7178, Perplexity: 2692380942323445042691593046766830443968783677039305317669871611870857856644433121916297639266804293546256625571004126234449917796273469952603080588175530930619736870590455964906818088910806856567483269120.00, Time: 611.09


TypeError: ignored